In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import plotly.express as px

In [3]:
df = pd.read_csv("/Users/jason/Documents (Mac)/DE data/filtered_data.csv")

/var/folders/4r/0r_9q19d30d02mbxyh2d_m7w0000gn/T/ipykernel_58655/1093374122.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/jason/Documents (Mac)/DE data/filtered_data.csv")


In [37]:
print("\n--- Dataset Info ---")
df.info()

# Display basic statistics of numerical columns
print("\n--- Numerical Statistics ---")
print(df.describe())

# Display the first 20 rows
print("\n--- First Last 20 Rows ---")
print(df.head(20))
print(df.tail(20))


--- Dataset Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52695282 entries, 0 to 52695281
Data columns (total 11 columns):
 #   Column               Dtype  
---  ------               -----  
 0   time_id              object 
 1   transit_mode         object 
 2   station_complex_id   object 
 3   station_complex      object 
 4   borough              object 
 5   payment_method       object 
 6   fare_class_category  int64  
 7   ridership            int64  
 8   transfers            int64  
 9   latitude             float64
 10  longitude            float64
dtypes: float64(2), int64(3), object(6)
memory usage: 4.3+ GB

--- Numerical Statistics ---
       fare_class_category     ridership     transfers      latitude  \
count         5.269528e+07  5.269528e+07  5.269528e+07  5.269528e+07   
mean          4.273945e+00  4.430394e+01  1.860902e+00  4.073229e+01   
std           2.289593e+00  1.415217e+02  1.151619e+01  7.834887e-02   
min           1.000000e+00  1.000000e+00 

In [38]:
# Drop specified columns and keep all others
_df = df.drop(['station_complex', 'latitude', 'longitude'], axis=1)

# Save to CSV
_df.to_csv('filtered_final.csv', index=False)

print("Columns in the new DataFrame:")
print(_df.columns.tolist())

Columns in the new DataFrame:
['station_complex_id', 'station_complex', 'latitude', 'longitude']


In [6]:

df['transit_timestamp'] = pd.to_datetime(df['transit_timestamp'])

# Then format it to the desired format
df['transit_timestamp'] = df['transit_timestamp'].dt.strftime('%Y-%m-%d-%H')

# Alternatively, you could also rename it to time_id if needed
df = df.rename(columns={'transit_timestamp': 'time_id'})

In [7]:
print("\n--- Dataset Info ---")
df.info()

# Display basic statistics of numerical columns
print("\n--- Numerical Statistics ---")
print(df.describe())

# Display the first 20 rows
print("\n--- First Last 20 Rows ---")
print(df.head(20))
print(df.tail(20))


--- Dataset Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52695282 entries, 0 to 52695281
Data columns (total 11 columns):
 #   Column               Dtype  
---  ------               -----  
 0   time_id              object 
 1   transit_mode         object 
 2   station_complex_id   object 
 3   station_complex      object 
 4   borough              object 
 5   payment_method       object 
 6   fare_class_category  object 
 7   ridership            int64  
 8   transfers            int64  
 9   latitude             float64
 10  longitude            float64
dtypes: float64(2), int64(2), object(7)
memory usage: 4.3+ GB

--- Numerical Statistics ---
          ridership     transfers      latitude     longitude
count  5.269528e+07  5.269528e+07  5.269528e+07  5.269528e+07
mean   4.430394e+01  1.860902e+00  4.073229e+01 -7.393515e+01
std    1.415217e+02  1.151619e+01  7.834887e-02  5.616036e-02
min    1.000000e+00  0.000000e+00  4.057613e+01 -7.407484e+01
25%    4.000000e+00

In [36]:
stations_df = stations_df[['station_complex_id', 'station_complex', 'latitude', 'longitude']].drop_duplicates(
    subset=['station_complex_id'],
    keep='first'  )

print("First few rows after removing duplicates:")
print(stations_df.head())

print("\nTotal number of unique stations:", len(stations_df))

# Verify no duplicates remain
print("\nNumber of duplicates remaining:", 
    stations_df['station_complex_id'].duplicated().sum())

First few rows after removing duplicates:
  station_complex_id             station_complex   latitude  longitude
0                209    Far Rockaway-Mott Av (A)  40.603996 -73.755400
1                221      21 St-Queensbridge (F)  40.754204 -73.942830
2                110               Forest Av (M)  40.704422 -73.903076
3                177  Clinton-Washington Avs (C)  40.683260 -73.965836
4                434           3 Av-149 St (2,5)  40.816110 -73.917755

Total number of unique stations: 854

Number of duplicates remaining: 0


In [ ]:
stations_df = df[[
    'station_complex_id',
    'station_complex',
    'latitude',
    'longitude'
]].drop_duplicates()

# Reset the index to have clean sequential indices
stations_df = stations_df.reset_index(drop=True)




# Create stations dataframe by grouping and taking mean of coordinates
stations_df = df.groupby(['station_complex_id', 'station_complex']).agg({
    'latitude': 'mean',  # takes average of latitude
    'longitude': 'mean'  # takes average of longitude
}).reset_index()

# Round coordinates to 6 decimal places for consistency
stations_df['latitude'] = stations_df['latitude'].round(6)
stations_df['longitude'] = stations_df['longitude'].round(6)

In [34]:


# Keep only the first occurrence of each station
stations_df = stations_df[['station_complex_id', 'station_complex', 'latitude', 'longitude']].drop_duplicates(
    subset=['station_complex_id', 'station_complex'],
    keep='first'
)

# Show the results
print("First few rows of cleaned stations data:")
print(stations_df.head())

# First show how many duplicates we have
print("Before cleaning - number of duplicate stations:")
print(df[['station_complex_id', 'station_complex']].duplicated().sum())



# Reset the index
stations_df = stations_df.reset_index(drop=True)

# Show the results
print("\nAfter cleaning - first few rows:")
print(stations_df.head())

print("\nTotal number of unique stations:", len(stations_df))

stations_df.to_csv('unique_stations.csv', index=False)


First few rows of cleaned stations data:
  station_complex_id             station_complex   latitude  longitude
0                209    Far Rockaway-Mott Av (A)  40.603996 -73.755400
1                221      21 St-Queensbridge (F)  40.754204 -73.942830
2                110               Forest Av (M)  40.704422 -73.903076
3                177  Clinton-Washington Avs (C)  40.683260 -73.965836
4                434           3 Av-149 St (2,5)  40.816110 -73.917755
Before cleaning - number of duplicate stations:
52694428

After cleaning - first few rows:
  station_complex_id             station_complex   latitude  longitude
0                209    Far Rockaway-Mott Av (A)  40.603996 -73.755400
1                221      21 St-Queensbridge (F)  40.754204 -73.942830
2                110               Forest Av (M)  40.704422 -73.903076
3                177  Clinton-Washington Avs (C)  40.683260 -73.965836
4                434           3 Av-149 St (2,5)  40.816110 -73.917755

Total number of

In [11]:
# To see just the unique categories
unique_fares = df['fare_class_category'].unique()
print("Unique fare categories:")
print(unique_fares)

# To see unique categories and their counts
fare_counts = df['fare_class_category'].value_counts()
print("\nFare category counts:")
print(fare_counts)

# For a more detailed summary
fare_summary = df['fare_class_category'].value_counts(normalize=True).reset_index()
fare_summary.columns = ['Category', 'Percentage']
fare_summary['Count'] = df['fare_class_category'].value_counts().values
print("\nDetailed summary:")
print(fare_summary)

# To just get the number of different categories
num_categories = df['fare_class_category'].nunique()
print(f"\nTotal number of different fare categories: {num_categories}")

Unique fare categories:
['Metrocard - Other' 'Metrocard - Fair Fare' 'Metrocard - Full Fare'
 'Metrocard - Seniors & Disability' 'Metrocard - Unlimited 30-Day'
 'Metrocard - Unlimited 7-Day' 'OMNY - Full Fare' 'Metrocard - Students'
 'OMNY - Other' 'OMNY - Seniors & Disability']

Fare category counts:
fare_class_category
OMNY - Full Fare                    7351526
Metrocard - Full Fare               7310318
Metrocard - Unlimited 7-Day         7178211
Metrocard - Unlimited 30-Day        6940959
Metrocard - Other                   6742357
Metrocard - Fair Fare               6392130
Metrocard - Seniors & Disability    5860018
Metrocard - Students                3574148
OMNY - Seniors & Disability         1187924
OMNY - Other                         157691
Name: count, dtype: int64

Detailed summary:
                           Category  Percentage    Count
0                  OMNY - Full Fare    0.139510  7351526
1             Metrocard - Full Fare    0.138728  7310318
2       Metrocard - U

In [24]:
fare_categories = pd.DataFrame({
    'fare_category_id': range(1, 11),  # Creates IDs from 1 to 10
    'fare_class_category': [
        'OMNY - Full Fare',
        'Metrocard - Full Fare',
        'Metrocard - Unlimited 7-Day',
        'Metrocard - Unlimited 30-Day',
        'Metrocard - Other',
        'Metrocard - Fair Fare',
        'Metrocard - Seniors & Disability',
        'Metrocard - Students',
        'OMNY - Seniors & Disability',
        'OMNY - Other'
    ]
})

print("Fare Categories Table:")
print(fare_categories)

fare_categories.to_csv('fare_categories.csv', index=False)


Fare Categories Table:
   fare_category_id               fare_class_category
0                 1                  OMNY - Full Fare
1                 2             Metrocard - Full Fare
2                 3       Metrocard - Unlimited 7-Day
3                 4      Metrocard - Unlimited 30-Day
4                 5                 Metrocard - Other
5                 6             Metrocard - Fair Fare
6                 7  Metrocard - Seniors & Disability
7                 8              Metrocard - Students
8                 9       OMNY - Seniors & Disability
9                10                      OMNY - Other


In [13]:
# Create a dictionary mapping from category names to IDs
fare_category_map = {
    'OMNY - Full Fare': 1,
    'Metrocard - Full Fare': 2,
    'Metrocard - Unlimited 7-Day': 3,
    'Metrocard - Unlimited 30-Day': 4,
    'Metrocard - Other': 5,
    'Metrocard - Fair Fare': 6,
    'Metrocard - Seniors & Disability': 7,
    'Metrocard - Students': 8,
    'OMNY - Seniors & Disability': 9,
    'OMNY - Other': 10
}

# Replace the categories with IDs in the main dataframe
df['fare_class_category'] = df['fare_class_category'].map(fare_category_map)

# To verify the change worked
print("First few rows after replacement:")
print(df[['fare_class_category']].head())

# To see value counts of the IDs
print("\nCounts of fare category IDs:")
print(df['fare_class_category'].value_counts().sort_index())

First few rows after replacement:
   fare_class_category
0                    5
1                    6
2                    2
3                    2
4                    7

Counts of fare category IDs:
fare_class_category
1     7351526
2     7310318
3     7178211
4     6940959
5     6742357
6     6392130
7     5860018
8     3574148
9     1187924
10     157691
Name: count, dtype: int64


In [14]:
print(df.head(20))
print(df.tail(20))

          time_id transit_mode station_complex_id  \
0   2024-06-30-10       subway                209   
1   2024-06-30-10       subway                221   
2   2024-06-30-10       subway                110   
3   2024-06-30-10       subway                177   
4   2024-06-30-10       subway                434   
5   2024-06-30-10       subway                155   
6   2024-06-30-10       subway                208   
7   2024-06-30-10       subway                210   
8   2024-06-30-10       subway                151   
9   2024-06-30-10       subway                114   
10  2024-06-30-10       subway                119   
11  2024-06-30-10       subway                216   
12  2024-06-30-10       subway                123   
13  2024-06-30-10       subway                141   
14  2024-06-30-10       subway                 65   
15  2024-06-30-10       subway                624   
16  2024-06-30-10       subway                436   
17  2024-06-30-10       subway                

In [39]:
df = pd.read_csv("MTA-filtered_final.csv")

print("\n--- Dataset Info ---")
df.info()

# Display basic statistics of numerical columns
print("\n--- Numerical Statistics ---")
print(df.describe())

# Display the first 20 rows
print("\n--- First Last 20 Rows ---")
print(df.head(20))
print(df.tail(20))

/var/folders/4r/0r_9q19d30d02mbxyh2d_m7w0000gn/T/ipykernel_58655/2852576817.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("MTA-filtered_final.csv")



--- Dataset Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52695282 entries, 0 to 52695281
Data columns (total 8 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   time_id              object
 1   transit_mode         object
 2   station_complex_id   object
 3   borough              object
 4   payment_method       object
 5   fare_class_category  int64 
 6   ridership            int64 
 7   transfers            int64 
dtypes: int64(3), object(5)
memory usage: 3.1+ GB

--- Numerical Statistics ---
       fare_class_category     ridership     transfers
count         5.269528e+07  5.269528e+07  5.269528e+07
mean          4.273945e+00  4.430394e+01  1.860902e+00
std           2.289593e+00  1.415217e+02  1.151619e+01
min           1.000000e+00  1.000000e+00  0.000000e+00
25%           2.000000e+00  4.000000e+00  0.000000e+00
50%           4.000000e+00  1.200000e+01  0.000000e+00
75%           6.000000e+00  3.600000e+01  1.000000e+00
max           